In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')

In [3]:
train_data_labels = np.array(train_data['label']) # labels
train_data_pixels_1d = np.array(train_data.drop(columns=['label'], axis=1))   # pixels

In [4]:
# normalize data
train_data_pixels_1d = train_data_pixels_1d / 255.0
test_data = test_data / 255.0

In [5]:
print(train_data_pixels_1d.shape)
print(test_data.shape)

(42000, 784)
(28000, 784)


In [6]:
train_data_pixels_2d = train_data_pixels_1d.reshape(train_data_pixels_1d.shape[0], 28, 28, 1)
test_data_2d = np.array(test_data).reshape(test_data.shape[0], 28, 28, 1)

In [7]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
# make label into onehot array
one_hot_label = to_categorical(train_data_labels)

Using TensorFlow backend.


In [8]:
from sklearn.model_selection import train_test_split

In [11]:
train_X, test_X, train_y, test_y = train_test_split(train_data_pixels_2d, one_hot_label, random_state=0)

In [14]:
train_data_pixels_2d

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [33]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D

In [34]:
# define model 
model = Sequential()

# conv2d 1
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
# conv2d 2
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# conv2d 3
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
# conv2d 5
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dense(256, activation = "relu"))
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [35]:
# Define the optimizer
from keras.optimizers import RMSprop
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
# Compile the model
model.compile(optimizer = 'adam' , loss = "categorical_crossentropy", metrics=["accuracy"])

In [42]:
batch_size = 86
epochs = 10
model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, validation_data=(test_X, test_y))

Train on 31500 samples, validate on 10500 samples
Epoch 1/10
31500/31500 [==============================] - 157s 5ms/step - loss: 0.0375 - acc: 0.9897 - val_loss: 0.0381 - val_acc: 0.9898
Epoch 2/10
31500/31500 [==============================] - 160s 5ms/step - loss: 0.0310 - acc: 0.9910 - val_loss: 0.0324 - val_acc: 0.9908
Epoch 3/10
31500/31500 [==============================] - 164s 5ms/step - loss: 0.0275 - acc: 0.9920 - val_loss: 0.0406 - val_acc: 0.9897
Epoch 4/10
31500/31500 [==============================] - 159s 5ms/step - loss: 0.0283 - acc: 0.9924 - val_loss: 0.0491 - val_acc: 0.9890
Epoch 5/10
31500/31500 [==============================] - 164s 5ms/step - loss: 0.0221 - acc: 0.9940 - val_loss: 0.0428 - val_acc: 0.9912
Epoch 6/10
31500/31500 [==============================] - 159s 5ms/step - loss: 0.0198 - acc: 0.9943 - val_loss: 0.0407 - val_acc: 0.9902
Epoch 7/10
31500/31500 [==============================] - 159s 5ms/step - loss: 0.0222 - acc: 0.9937 - val_loss: 0.0400 - 

In [43]:
model.save('mnist_conv2d_layer.model')

# Make Submission

In [44]:
train_data_predictions = model.predict(test_data_2d)
train_data_predictions_numbers = [np.argmax(pred) for pred in train_data_predictions]

In [45]:
submission_df = pd.DataFrame(columns=['ImageId', 'Label'], index=range(len(train_data_predictions)))
submission_df['Label'] = train_data_predictions_numbers
submission_df['ImageId'] = range(1, len(train_data_predictions)+1)

In [46]:
submission_df.to_csv('./data/submission_conv2d.csv', index=False)

In [47]:
submission_df.tail()

,ImageId,Label
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
27999,28000,2
